In [1]:
# モデルのテストと、sagemakerのデプロイを行うファイル
%ls
%pwd

custombdd/                                modules/
Faster_R_CNN_ver_sagemaker_moduled.ipynb  requirements.txt
models/                                   sagemaker_entry_point.py
model.tar.gz                              testInf.jpg


'/home/ec2-user/SageMaker/fasterRcnn/colab_frcnn'

In [2]:
!nvidia-smi

Tue Jun  6 08:08:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   47C    P8    15W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from modules.infer import testShowInferenceImage
from modules.train import train
from modules.MLparamaters import getMLParamaters


In [4]:

# print(torch.__version__)
# print(torchvision.__version__)
# print(torchtext.__version__)
# print(torchaudio.__version__)
# 1.13.1
# 0.14.1
# 0.14.1
# 0.13.1

In [ ]:

! echo checkpoint1

# ここ、pip installは初回のみでいいかも
!pip install -r requirements.txt
%pwd


In [ ]:
import torch
#学習済みモデルで推論する場合
paramaters = getMLParamaters()

# train()

use_model=torch.load(paramaters.path+'/models/model.pt')

testShowInferenceImage(use_model)

In [7]:
# !tar zcvf model.tar.gz models/model.pt

models/model.pt


In [ ]:
# sagamakerでエンドポイントデプロイ関連
import sagemaker


from sagemaker.pytorch import PyTorch

sagemaker.__version__

from sagemaker import get_execution_role
role = get_execution_role()
print(role)



s3_path="s3://mlearning-bucket/sageMakerDeployTest/model/model.tar.gz"

hyper_param = {
    'epochs':100,
    'batch-size': 100,
    'lr': 0.01,
    'momentum': 0.9,
}

pytorch_model = PyTorch(model_data=s3_path,
                            entry_point="sagemaker_entry_point.py",
                            hyperparameters=hyper_param,
                            role=role,
                            framework_version='1.2.0',
                            py_version='py3',
                            train_instance_count=2,
                            train_instance_type='ml.c5.xlarge')
deploy_params = {
    'instance_type'          : 'ml.t2.medium',
    'initial_instance_count' : 1
}

# デプロイ
predictor = pytorch_model.deploy(**deploy_params)

arn:aws:iam::958305726855:role/service-role/AmazonSageMaker-ExecutionRole-20230406T155831


In [ ]:
# デプロイしたものの画像表示テスト

import matplotlib.pyplot as plt
import numpy as np
import cv2

filename = 'testInf.jpg'

# インライン表示
%matplotlib inline

# Read image into memory
payload = None
with open(filename, 'rb') as f:
    payload = f.read()

#画像の表示
frombuff = np.frombuffer(payload, dtype='uint8') # byteをnumpyにしている(1次元)

input_data = cv2.imdecode(frombuff, cv2.IMREAD_UNCHANGED) # numpyを画像の形式にdecodeしている。

bytearray_data = bytearray(payload)
plt.imshow(input_data) 
plt.show()

from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = IdentitySerializer()
predictor.deserializer = JSONDeserializer()
results = predictor.predict(bytearray_data, initial_args={'ContentType': 'application/x-image'})

print(results)

In [ ]:
# ログ書き出しテスト

import json
import time
import datetime
import boto3

client = boto3.client('logs')

def get_log_stream_token(log_group_name, log_stream_name):
    # ログイベントがログストリームに存在する場合、書き込むを行うにはログストリームのトークンが必要なのでそれを取得する
    response = client.describe_log_streams(
        logGroupName=log_group_name,
        logStreamNamePrefix=log_stream_name,
    )
    log_streams = response['logStreams']
    log_stream = log_streams[0]
    log_stream_token = log_stream['uploadSequenceToken']
    return log_stream_token

def put_custom_log_event(log_group_name, log_stream_name, log_stream_token):
    # ログストリームのトークンを元にログイベントを書き込む
    response = client.put_log_events(
        logGroupName=log_group_name,
        logStreamName=log_stream_name,
        logEvents=[
            {
                'timestamp': int(time.time()) * 1000,
                'message': 'testほげ'
            },
        ],
        sequenceToken=log_stream_token
      )

def main():
    log_group_name = '/aws/sagemaker/Endpoints/sagemakerinftest3'
    log_stream_name = 'AllTraffic/i-03f1ee612f0e1aa0a'
    log_stream_token= get_log_stream_token(log_group_name, log_stream_name)
    put_custom_log_event(log_group_name, log_stream_name, log_stream_token)

main()